# Importing required packages

In [ ]:
import pandas as pd
import numpy as np
import requests
import tweepy
import os
import seaborn as sns
import json
import matplotlib.pyplot as plt

### Reading dataset and printing first two rows

In [ ]:
df_twitter_archive =  pd.read_csv('twitter-archive-enhanced.csv')
df_twitter_archive.head(2)

#### Gathering image prediction file on udacity hosted server 

Using python request --

In [ ]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
open('image_predictions.tsv', 'wb').write(response.content)

In [ ]:
df_image_prediction = pd.read_csv('image_predictions.tsv',sep='\t')
df_image_prediction.head(2)

## Stop running from here api call ahead will take 30-40 mins .

### Gathering twitter data throught tweepy : 

In [ ]:
CONSUMER_KEY = '5Uur0mo4ol2kB8yhtZ1VxXS0u'
CONSUMER_SECRET = 'h8E7fSpXWiMoBel7G1ZOAeu4Mgru0v0MtxH5ehYE1RKM89SiBH'
OAUTH_TOKEN = 'ct9aNnU0FQR0UKJVn1i1W3Y8omqSewiQWUcRaygB'
OAUTH_TOKEN_SECRET ='D3qslrbdOU5fqTOp951kOIuZbkeTPBodnjNYoEGFR63Ft'


auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

api = tweepy.API(auth, 
                 parser = tweepy.parsers.JSONParser(), 
                 wait_on_rate_limit = True, 
                 wait_on_rate_limit_notify = True)


#Download Tweepy status object based on Tweet ID and store in list
# Fetch tweets from the twitter API using the following loop:
list_of_tweets = []
# Tweets that can't be found are saved in the list below:
cant_find_tweets_for_those_ids = []
for each_id in my_ids:   
    try:
        list_of_tweets.append(api.get_status(each_id))
    except Exception as e:
        cant_find_tweets_for_those_ids.append(each_id)

In [ ]:
# check data in list : 

print("The list of tweets" ,len(list_of_tweets))
print("The list of tweets no found" , len(cant_find_tweets_for_those_ids))

In [ ]:
#Then in this code block we isolate the json part of each tweepy 
#status object that we have downloaded and we add them all into a list

my_list_of_dicts = []
for each_json_tweet in list_of_tweets:
    my_list_of_dicts.append(each_json_tweet)

In [ ]:
with open('tweet_json.txt', 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))

In [ ]:
#identify information of interest from JSON dictionaries in txt file
#and put it in a dataframe called tweet JSON
my_demo_list = []
with open('tweet_json.txt',encoding ='utf-8') as json_file:
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        whole_tweet = each_dictionary['text']
        only_url = whole_tweet[whole_tweet.find('https'):]
        favorite_count = each_dictionary['favorite_count']
        retweet_count = each_dictionary['retweet_count']
        followers_count = each_dictionary['user']['followers_count']
        friends_count = each_dictionary['user']['friends_count']
        whole_source = each_dictionary['source']
        only_device = whole_source[whole_source.find('rel="nofollow">') + 15:-4]
        source = only_device
        retweeted_status = each_dictionary['retweeted_status'] = each_dictionary.get('retweeted_status', 'Original tweet')
        if retweeted_status == 'Original tweet':
            url = only_url
        else:
            retweeted_status = 'This is a retweet'
            url = 'This is a retweet'
        my_demo_list.append({
            'tweet_id' : str(tweet_id),
            'favorite_count' : int(favorite_count),
            'retweet_count' : int(retweet_count),
            'followers_count' : int(followers_count),
            'friends_count' : int(friends_count),
            'source' : source,
            'retweeted_status' : retweeted_status,
            'url' : url,
            
        })
tweet_json = pd.DataFrame(my_demo_list, columns = ['tweet_id', 'favorite_count','retweet_count','followers_count', 'friends_count','source','retweeted_status', 'url'])

## Start Running from here now : 

In [ ]:
tweet_json = pd.read_csv('tweet_json.csv')
tweet_json.head(2)

#### Saving dataframe tweet_json to a csv file so that you dont have to run api call again when you rerun the code : 

In [ ]:
tweet_json.to_csv('tweet_json.csv',index=False)

## Assesing data for issues : 
#### Visual assesment / Programatic assesment

## Assessing tweet_json data

In [ ]:
tweet_json.head(2)

In [ ]:
tweet_json.info()

In [ ]:
tweet_json.describe()

In [ ]:
tweet_json.sample(5)

In [ ]:
sum(tweet_json.duplicated())

## Assesing image prediction data 

In [ ]:
df_image_prediction.head(2)

In [ ]:
df_image_prediction.info()

In [ ]:
df_image_prediction.describe()

In [ ]:
df_image_prediction.sample(5)

In [ ]:
sum(df_image_prediction.jpg_url.duplicated())

In [ ]:
df_image_prediction.p1.value_counts()

In [ ]:
df_image_prediction.p1.value_counts()

In [ ]:
df_image_prediction.img_num.value_counts()

In [ ]:
df_image_prediction.p1_conf.mean()

In [ ]:
df_image_prediction.p2_conf.mean()

In [ ]:
sum(df_image_prediction.tweet_id.duplicated())

In [ ]:
df_image_prediction.isnull().sum()

## Assesing twitter archive data

In [ ]:
df_twitter_archive.head(2)

In [ ]:
df_twitter_archive.info()

In [ ]:
df_twitter_archive.describe()

In [ ]:
df_twitter_archive.head(3)

In [ ]:
df_twitter_archive.columns

In [ ]:
df_twitter_archive.isnull().sum()

In [ ]:
sum(df_twitter_archive['tweet_id'].duplicated())

In [ ]:
df_twitter_archive['source'][0]

In [ ]:
df_twitter_archive['source'][0]

In [ ]:
# Checking all different values of rating numerator
df_twitter_archive.rating_numerator.value_counts()

##### Checking each value one by one for irregularities :

In [ ]:
# selecting some random values and inspecting them for reason of irregularitits
print(df_twitter_archive.loc[df_twitter_archive.rating_numerator == 204, 'text']) 
print(df_twitter_archive.loc[df_twitter_archive.rating_numerator == 143, 'text']) 
print(df_twitter_archive.loc[df_twitter_archive.rating_numerator == 666, 'text']) 
print(df_twitter_archive.loc[df_twitter_archive.rating_numerator == 1176, 'text'])
print(df_twitter_archive.loc[df_twitter_archive.rating_numerator == 144, 'text'])

In [ ]:
#print whole text in order to verify numerators and denominators
print(df_twitter_archive['text'][1120]) #17 dogs
print(df_twitter_archive['text'][1634]) #13 dogs
print(df_twitter_archive['text'][313]) #just a tweet to explain actual ratings, this will be ignored when cleaning data
print(df_twitter_archive['text'][189]) #no picture, this will be ignored when cleaning data
print(df_twitter_archive['text'][1779]) #12 dogs

In [ ]:
# Checking all different values of rating denominator

df_twitter_archive.rating_denominator.value_counts()

In [ ]:
print(df_twitter_archive.loc[df_twitter_archive.rating_denominator == 11, 'text'])
print(df_twitter_archive.loc[df_twitter_archive.rating_denominator == 2, 'text'])
print(df_twitter_archive.loc[df_twitter_archive.rating_denominator == 16, 'text'])
print(df_twitter_archive.loc[df_twitter_archive.rating_denominator == 110, 'text'])
print(df_twitter_archive.loc[df_twitter_archive.rating_denominator == 15, 'text'])
print(df_twitter_archive.loc[df_twitter_archive.rating_denominator == 90, 'text'])
print(df_twitter_archive.loc[df_twitter_archive.rating_denominator == 130, 'text'])
print(df_twitter_archive.loc[df_twitter_archive.rating_denominator == 7, 'text'])

In [ ]:
df_twitter_archive.iloc[2335]

In [ ]:
print(df_twitter_archive['text'][784]) # Retweet -- will be deleted
print(df_twitter_archive['text'][1068]) # Actual rating 14/10 manual change
print(df_twitter_archive['text'][1662]) # Actual rating 10/10 manual change
print(df_twitter_archive['text'][2335]) # Actual rating 9/10 manual change
print(df_twitter_archive['text'][1663]) # Tweet to explain rating
print(df_twitter_archive['text'][1635]) # 11 dogs

#### by exploring (Reviewer point )i found that the data has some wrong numerators and denominators in rating that i will change manually 

In [ ]:
with pd.option_context('max_colwidth', 200):
    display(df_twitter_archive[df_twitter_archive['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

#### here we can see that rating are wrong we will have to change them also manually 

# Cleaning : 
    Quality : 
        df_twitter_archive : 
            1. Delete columns that wont be used for analysis(in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls)
            2. Separate timestamp into day - month - year (3 columns)
            3. Delete retweets.
            4. Correct numerators and denominators
        df_image_prediction :
            1. Drop 66 jpg_url duplicated
            2. Create 1 column for image prediction and 1 column for confidence level
            3. Delete columns that won't be used for analysis 
            
        tweet_json :
            1. Keep original tweets only
            
    Tidiness : 
        df_twitter_archive:
            1. Melt dogoo puppo to one col dog statge
            2. Change tweet_id to type int64 in order to merge with the other 2 tables
            3. All tables should be part of one dataset

### Making a copy of dataframes before cleaning

In [ ]:
twitter_archive_clean = df_twitter_archive.copy()
image_prediction_clean = df_image_prediction.copy()
tweet_json_clean = tweet_json.copy()

#### cleaning twitter_archive

df_twitter_archive : 
        1. Delete columns that wont be used for analysis(in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls)
        2. Separate timestamp into day - month - year (3 columns)
        3. Delete retweets.
        4. Correct numerators and denominators

In [ ]:
df_twitter_archive.retweeted_status_id.value_counts().count()

1. Twitter archive - Delete retweets .

Based on info, there are 181 values in retweeted_status_id and retweeted_status_user_id. Delete the retweets. Once I merge twitter_archive and image_prediction, I will only keep the ones with images.

-- Keeping only original tweets deleting all retweets

In [ ]:
twitter_archive_clean = twitter_archive_clean[pd.isnull(twitter_archive_clean['retweeted_status_user_id'])]

2. Twitter archive - Delete columns that won't be used for analysis

In [ ]:
#CODE: Delete columns no needed
twitter_archive_clean.drop(['source','in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'],1,inplace= True)

In [ ]:
print(list(twitter_archive_clean))

Tidiness : 
1. Twitter_archive - Melt (doggo, floofer, pupper and puppo columns) to one col dog statge 

In [ ]:
twitter_archive_clean = pd.melt(twitter_archive_clean,id_vars = ['tweet_id','timestamp','text','rating_numerator','rating_denominator','name'],var_name = 'dogs',value_name = 'dogs_stage')

In [ ]:
# Drop dogs
twitter_archive_clean = twitter_archive_clean.drop('dogs', 1)

# Sort by dogs_stage then drop duplicated based on tweet_id except the last occurrence
twitter_archive_clean = twitter_archive_clean.sort_values('dogs_stage').drop_duplicates(subset='tweet_id', 
                                                                                        keep='last')

In [ ]:
# Test :
twitter_archive_clean['dogs_stage'].value_counts()

In [ ]:
twitter_archive_clean.head()

### Indices are not ordered 

In [ ]:
# indices = np.arange(twitter_archive_clean.shape[0])
# twitter_archive_clean.index = indices

4. Twitter_archive - Separate timestamp into day - month - year (3 columns)

First convert timestamp to datetime. Then extract year, month and day to new columns. Finally drop timestamp column.

In [ ]:
# convert timestamp to datetime
twitter_archive_clean.timestamp = pd.to_datetime(twitter_archive_clean.timestamp)

In [ ]:
#extract year, month and day to new columns

twitter_archive_clean["day"] = twitter_archive_clean['timestamp'].map(lambda x: x.day)
twitter_archive_clean["month"] = twitter_archive_clean['timestamp'].map(lambda x: x.month)
twitter_archive_clean["year"] = twitter_archive_clean['timestamp'].map(lambda x: x.year)

In [ ]:
# Test :
twitter_archive_clean.sample(5)

In [ ]:
# Drop the timestamp col not needed anymore
twitter_archive_clean.drop('timestamp',1,inplace = True)

In [ ]:
# Test
twitter_archive_clean.sample(5)

5. Twitter_archive - Correc numerators 

Steps : 
1. first we will correct numerator that have wrong rating (wrong data extracted)
2. then we will correct dog rating in which there are multiple dogs and that are having rating denominators greater than 10 
3. then we will correct their denominators
also 

> first we will correct numerator that have wrong rating (wrong data extracted)

In [ ]:
# Checking for wrong data : 

with pd.option_context('max_colwidth', 200):
    display(twitter_archive_clean[twitter_archive_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

In [ ]:
# Converting datatype to float to accept float values
twitter_archive_clean[['rating_numerator', 'rating_denominator']] = twitter_archive_clean[['rating_numerator','rating_denominator']].astype(float)

In [ ]:
# Updating numerators manually: 

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 883482846933004288), 'rating_numerator'] = 13.5
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 786709082849828864), 'rating_numerator'] = 9.75
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 778027034220126208), 'rating_numerator'] = 11.27
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 681340665377193984), 'rating_numerator'] = 9.5
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 680494726643068929), 'rating_numerator'] = 11.26

# Testing code : 
with pd.option_context('max_colwidth', 200):
    display(twitter_archive_clean[twitter_archive_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

Twitter_archive - Correc denominators

a. manual cleaning of columns 

Five tweets with denominator not equal to 10 for special circunstances. Update both numerators and denominators when necessary. Delete other five tweets because they do not have actual ratings.

In [ ]:
# Checking again before cleaning:
print(df_twitter_archive['text'][784]) # Retweet -- will be deleted
print(df_twitter_archive['text'][1068]) # Actual rating 14/10 manual change
print(df_twitter_archive['text'][1662]) # Actual rating 10/10 manual change
print(df_twitter_archive['text'][2335]) # Actual rating 9/10 manual change
print(df_twitter_archive['text'][1663]) # Tweet to explain rating
print(df_twitter_archive['text'][1635]) # 11 dogs

In [ ]:
# extracting its tweet id to clean better
# not verifying from cleaning dataset because its not indexed properly 
print(df_twitter_archive['tweet_id'][1068])

In [ ]:
#CODE: Update both numerators and denominators
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 740373189193256964), 'rating_numerator'] = 14
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 740373189193256964), 'rating_denominator'] = 10

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 682962037429899265), 'rating_numerator'] = 10
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 682962037429899265), 'rating_denominator'] = 10

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 666287406224695296), 'rating_numerator'] = 9
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 666287406224695296), 'rating_denominator'] = 10

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 722974582966214656), 'rating_numerator'] = 13
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 722974582966214656), 'rating_denominator'] = 10

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 716439118184652801), 'rating_numerator'] = 13.5
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 716439118184652801), 'rating_denominator'] = 10

#CODE: Delete five tweets with no actual ratings
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'] != 832088576586297345]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'] != 810984652412424192]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'] != 682808988178739200]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'] != 835246439529840640]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'] != 686035780142297088]

#TEST: Left only the group dogs for programatically clean
with pd.option_context('max_colwidth', 200):
    display(twitter_archive_clean[twitter_archive_clean['rating_denominator'] != 10][['tweet_id',
                                                                                      'text',
                                                                                      'rating_numerator',
                                                                                      'rating_denominator']])

> These tweets with denominator not equal to 10 are multiple dogs. For example, tweet_id 713900603437621000 has numerator and denominators 99/90 because there are 9 dogs in the picture https://t.co/mpvaVxKmc1.

In [ ]:
#CODE: Create a new column with rating in float type to avoid converting all int column to float
twitter_archive_clean['rating'] = 10 * twitter_archive_clean['rating_numerator'] / twitter_archive_clean['rating_denominator'].astype(float)

#TEST
twitter_archive_clean.sample(5)

### Cleaning image_prediction
 df_image_prediction :
        1. Drop 66 jpg_url duplicated
        2. Create 1 column for image prediction and 1 column for confidence level
        3. Delete columns that won't be used for analysis 

In [ ]:
image_prediction_clean = image_prediction_clean.drop_duplicates(subset=['jpg_url'], keep='last')

In [ ]:
image_prediction_clean.jpg_url.duplicated().sum()

In [ ]:
image_prediction_clean.head(5)

2. Image_prediction - Create 1 column for image prediction and 1 column for confidence level

Create a function where I keep the first true prediction along the confidence level as new columns.


In [ ]:
#CODE: the first true prediction (p1, p2 or p3) will be store in these lists
dog_type = []
confidence_list = []

#create a function with nested if to capture the dog type and confidence level
# from the first 'true' prediction
def image(image_prediction_clean):
    if image_prediction_clean['p1_dog'] == True:
        dog_type.append(image_prediction_clean['p1'])
        confidence_list.append(image_prediction_clean['p1_conf'])
    elif image_prediction_clean['p2_dog'] == True:
        dog_type.append(image_prediction_clean['p2'])
        confidence_list.append(image_prediction_clean['p2_conf'])
    elif image_prediction_clean['p3_dog'] == True:
        dog_type.append(image_prediction_clean['p3'])
        confidence_list.append(image_prediction_clean['p3_conf'])
    else:
        dog_type.append('Error')
        confidence_list.append('Error')

#series objects having index the image_prediction_clean column.        
image_prediction_clean.apply(image, axis=1)

#create new columns
image_prediction_clean['dog_type'] = dog_type
image_prediction_clean['confidence_list'] = confidence_list

In [ ]:
#drop rows that has prediction_list 'error'
image_prediction_clean = image_prediction_clean[image_prediction_clean['dog_type'] != 'Error']

#TEST: 
image_prediction_clean.info()

In [ ]:
#  Delete columns that won't be used for analysis

#CODE: print list of image_prediction columns
print(list(image_prediction_clean))

#Delete columns
image_prediction_clean = image_prediction_clean.drop(['img_num', 'p1', 
                                                      'p1_conf', 'p1_dog', 
                                                      'p2', 'p2_conf', 
                                                      'p2_dog', 'p3', 
                                                      'p3_conf', 
                                                      'p3_dog'], 1)

#TEST
list(image_prediction_clean)

In [ ]:
# Tweet_json - keep 2174 original tweets

#CODE:
tweet_json_clean = tweet_json_clean[tweet_json_clean['retweeted_status'] == 'Original tweet']

#TEST
tweet_json_clean['retweeted_status'].value_counts()

In [ ]:
# Tidiness - Change tweet_id to type int64 in order to merge with the other 2 tables



#CODE: change tweet_id from str to int
tweet_json_clean['tweet_id'] = tweet_json_clean['tweet_id'].astype(int)

#TEST
tweet_json_clean['tweet_id'].dtypes



In [ ]:
# Tidiness - All tables should be part of one dataset

#CODE: create a new dataframe that merge twitter_archive_clean and 
#image_prediction_clean
df_twitter1 = pd.merge(twitter_archive_clean, 
                      image_prediction_clean, 
                      how = 'left', on = ['tweet_id'])

#keep rows that have picture (jpg_url)
df_twitter1 = df_twitter1[df_twitter1['jpg_url'].notnull()]

#TEST
df_twitter1.info()

In [ ]:
df_twitter = pd.merge(df_twitter1, tweet_json_clean, 
                      how = 'left', on = ['tweet_id'])

#TEST
df_twitter.info()

In [ ]:
df_twitter['rating_numerator'].value_counts()

## Store the clean DataFrame in a CSV file

In [ ]:
df_twitter.to_csv('twitter_archive_master.csv', 
                 index=False, encoding = 'utf-8')

In [ ]:
df_twitter = pd.read_csv('twitter_archive_master.csv')

## Insights and visualisation 

In [ ]:
df_twitter['dog_type'].value_counts()

In [ ]:
df_dog_type = df_twitter.groupby('dog_type').filter(lambda x:len(x)>=25)

In [ ]:
order = df_dog_type['dog_type'].value_counts().index
color = sns.color_palette()[0]

In [ ]:
sns.countplot(data=df_dog_type,y='dog_type',order = order,color = color)
plt.xlabel("Count")
plt.ylabel('Type of dog')
plt.title('Histogram of the Most Rated Dog Type');

In [ ]:
df = df_twitter.groupby('dog_type').mean()
df.head()

### Highest rated dog type :  Bouvier_des_Flandres   
### Lowest rated dog type :  Japanese_spaniel               

In [ ]:
df = df_twitter.groupby(['dog_type'])['rating'].mean()
df = df.sort_values(ascending=False)
df = df[:10]

In [ ]:
df.head()

In [ ]:
df.plot(kind='bar',color=color,figsize=(10,6))

In [ ]:
df = df_twitter.groupby(['dog_type'])['rating'].mean()
df = df.sort_values(ascending=False)
df = df[-10:]

In [ ]:
df.plot(kind='bar',color=color,figsize=(10,6))

### Most common dog names :  

In [ ]:
df_twitter.name.value_counts()

In [ ]:
# Dropping irrelavent names.
df_twitter.drop(df_twitter[df_twitter['name'] == 'a'].index,inplace=True)

In [ ]:
df_twitter.name.value_counts()

In [ ]:
df_twitter.name.value_counts()[0:7].plot('barh', figsize=(10,8), title='Most Common Dog Names').set_xlabel("Number of Dogs");